In [1]:
# Initial Setup & Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [65]:
import pandas as pd

# Read in both CSV files
crypto_df = pd.read_csv("crypto_csv_with_weights.csv")
stock_df = pd.read_csv("stock_csv_with_weights.csv")

# Combine the two dataframes
combined_df = pd.concat([crypto_df, stock_df])

# Define a function to compute the weighted return for a given column
def weighted_return(group, col):
    return (group['Weight'] * group[col]).sum()

# Group by the timeframe column and calculate the weighted returns for each group
portfolio_returns = combined_df.groupby("Date").apply(
    lambda x: pd.Series({
        "portfolio_predicted_return": weighted_return(x, "Predicted_Return"),
        "portfolio_actual_return": weighted_return(x, "Actual_Return"),
        "portfolio_strategy_return": weighted_return(x, "Strategy_Return")
    })
).reset_index()

# Ensure that the DataFrame is sorted by timeframe for proper cumulative calculation
portfolio_returns = portfolio_returns.sort_values(by="Date")

# Calculate the cumulative portfolio strategy return
# Here, we assume that returns are expressed as decimals (e.g., 0.01 for 1%)
portfolio_returns["portfolio_strategy_cumulative_return"] = (
    (1 + portfolio_returns["portfolio_strategy_return"]).cumprod() - 1
) 

print(portfolio_returns)





                      Date  portfolio_predicted_return  \
0      2025-03-01 00:40:00                    0.000000   
1      2025-03-01 00:42:00                    0.000000   
2      2025-03-01 00:44:00                    0.000000   
3      2025-03-01 00:46:00                    0.000000   
4      2025-03-01 00:47:00                    0.000000   
...                    ...                         ...   
36913  2025-03-31 23:50:00                    0.000014   
36914  2025-03-31 23:52:00                    0.000004   
36915  2025-03-31 23:54:00                   -0.000026   
36916  2025-03-31 23:56:00                   -0.000032   
36917  2025-03-31 23:58:00                   -0.000036   

       portfolio_actual_return  portfolio_strategy_return  \
0                 0.000000e+00               0.000000e+00   
1                 0.000000e+00               0.000000e+00   
2                 0.000000e+00               0.000000e+00   
3                 0.000000e+00               0.000000e+00  

C:\Users\ASUS\AppData\Local\Temp\ipykernel_36568\1388521885.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  portfolio_returns = combined_df.groupby("Date").apply(


0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
36913   -0.033919
36914   -0.033919
36915   -0.033934
36916   -0.033071
36917   -0.037019
Name: portfolio_strategy_cumulative_return, Length: 36918, dtype: float64

In [58]:
crypto_df

,Unnamed: 0,Date,Stock_Close_Price,Predicted,Strategy_Return,Position,Cumulative_Return,Ticker,Rolling_Risk,Rolling_Average,Weight,Actual,Asset_Type
0,0,2025-03-01 00:47:00,84171.7000,0.000037,-0.001042,LONG,-0.001042,BTC,NaN,NaN,NaN,NaN,Crypto
1,34514,2025-03-01 00:47:00,2229.9000,-0.000087,-0.001296,SHORT,0.001296,ETH,NaN,NaN,NaN,NaN,Crypto
2,69028,2025-03-01 00:47:00,146.5100,0.000026,-0.001638,LONG,-0.001638,SOL,NaN,NaN,NaN,NaN,Crypto
3,103542,2025-03-01 00:47:00,2.1373,0.000039,-0.001825,LONG,-0.001825,XRP,NaN,NaN,NaN,NaN,Crypto
4,34515,2025-03-01 00:48:00,2227.0100,-0.000082,-0.000139,SHORT,0.001435,ETH,NaN,NaN,NaN,NaN,Crypto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138051,34512,2025-03-24 23:59:00,87498.1600,0.000049,-0.000864,LONG,0.032028,BTC,0.000594,0.000175,-9.539333e-09,NaN,Crypto
138052,69027,2025-03-25 00:00:00,2079.6800,-0.000091,0.000423,SHORT,0.533340,ETH,0.000856,0.000353,1.259822e-08,NaN,Crypto
138053,34513,2025-03-25 00:00:00,87422.5800,0.000047,0.000312,LONG,0.032350,BTC,0.000583,0.000222,-9.539333e-09,NaN,Crypto
138054,103541,2025-03-25 00:00:00,140.7600,-0.000019,0.000639,SHORT,0.737437,SOL,0.001053,0.000178,6.737286e-07,NaN,Crypto


In [40]:
def run_sensitivity_analysis(final_df):
    """
    Perform sensitivity analysis by adjusting the predicted returns and evaluating final cumulative return.
    Works with final_df.
    """
    factors = [0.5, 0.8, 0.9, 1.0, 1.1, 1.2, 1.5]
    results = []

    for factor in factors:
        # Adjust the predicted returns using the factor
        adjusted_predicted = final_df['Portfolio_Predicted_Return'] * factor

        # Determine position: LONG if predicted >= 0, else SHORT
        positions = np.where(adjusted_predicted >= 0, 1, -1)

        # Strategy return = actual return * position
        strategy_returns = positions * final_df['Portfolio_Actual_Return']

        # Compute cumulative return
        cumulative_returns = (1 + strategy_returns).cumprod()
        final_cum_return = cumulative_returns.iloc[-1]
        
        results.append({
            'Adjustment_Factor': factor,
            'Final_Cumulative_Return': final_cum_return
        })

    return pd.DataFrame(results)  

def run_stress_testing(final_df):
    """
    Perform stress testing by simulating market shocks on final_df.
    """

    scenarios = {
        'Crash': final_df['Portfolio_Actual_Return'] - 0.00005,  
        'Volatility_Spike': final_df['Portfolio_Actual_Return'] + np.random.normal(0, 0.00002, size=len(final_df)),
        'Prediction_Inverted': -final_df['Portfolio_Predicted_Return'],
        'Zero_Prediction': np.zeros_like(final_df['Portfolio_Predicted_Return']),
        'Normal': final_df['Portfolio_Actual_Return']
    }

    results = []

    for scenario_name, shock in scenarios.items():
        df_temp = final_df.copy()

        # Adjust predictions or actuals based on scenario
        if 'Prediction' in scenario_name or 'Zero' in scenario_name:
            df_temp['Adj_Predicted_Return'] = shock
            df_temp['Adj_Actual_Return'] = df_temp['Portfolio_Actual_Return']
        else:
            df_temp['Adj_Predicted_Return'] = df_temp['Portfolio_Predicted_Return']
            df_temp['Adj_Actual_Return'] = shock

        # Position based on adjusted prediction
        positions = np.where(df_temp['Adj_Predicted_Return'] > 0, 1, -1)

        # Strategy returns based on adjusted actual returns
        strategy_returns = positions * df_temp['Adj_Actual_Return']

        # Compute cumulative return
        cumulative_return = pd.Series(1 + strategy_returns).cumprod().iloc[-1]
        results.append((scenario_name, cumulative_return))

    return pd.DataFrame(results, columns=["Scenario", "Final_Cumulative_Return"])



In [50]:
final_df

,Date,Predicted_Return_crypto,Actual_Return_crypto,Predicted_Return_stock,Actual_Return_stock,Portfolio_Predicted_Return,Portfolio_Actual_Return
0,2025-03-01 00:40:00,NaN,NaN,-0.000010,0.000409,0.000000e+00,0.000000e+00
1,2025-03-01 00:40:00,NaN,NaN,-0.000005,0.000121,0.000000e+00,0.000000e+00
2,2025-03-01 00:42:00,NaN,NaN,-0.000004,0.000075,0.000000e+00,0.000000e+00
3,2025-03-01 00:42:00,NaN,NaN,-0.000010,0.000000,0.000000e+00,0.000000e+00
4,2025-03-01 00:44:00,NaN,NaN,-0.000004,0.000338,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...
311873,2025-03-31 23:58:00,NaN,NaN,-0.000007,-0.000175,-4.920282e-13,-1.159015e-11
311874,2025-03-31 23:58:00,NaN,NaN,-0.000005,0.000555,-1.254096e-05,1.387487e-03
311875,2025-03-31 23:58:00,NaN,NaN,-0.000005,0.000574,-3.181448e-13,3.795617e-11
311876,2025-03-31 23:58:00,NaN,NaN,0.000035,0.000084,2.305120e-11,5.537129e-11


In [49]:
stock_df[61950:]

,Date,Close_Price,Actual_Return,Predicted_Return,Position,Strategy_Return,Cumulative_Return,Ticker,Rolling_Risk,Rolling_Average,Weight
61950,2025-03-31 23:52:00,221.0800,0.000047,-0.000007,SHORT,-0.000047,1.202323,AAPL,0.000241,3.069654e-05,6.633264e-08
61951,2025-03-31 23:52:00,107.4199,-0.000286,-0.000005,SHORT,0.000286,1.168555,NVDA,0.000402,-5.327029e-05,6.611766e-08
61952,2025-03-31 23:54:00,572.8176,0.000365,-0.000045,SHORT,-0.000365,0.866197,META,0.000507,-3.279117e-04,-1.761221e-09
61953,2025-03-31 23:54:00,189.1600,0.000000,-0.000004,SHORT,-0.000000,1.341139,AMZN,0.000657,-2.495410e-04,2.499996e+00
61954,2025-03-31 23:54:00,256.1000,0.000010,0.000011,LONG,0.000010,1.007737,TSLA,0.000188,5.692499e-05,-1.500001e+00
61955,2025-03-31 23:54:00,107.3900,-0.000585,-0.000005,SHORT,0.000585,1.169238,NVDA,0.000431,-1.197701e-04,6.611766e-08
61956,2025-03-31 23:54:00,221.0800,0.000956,-0.000007,SHORT,-0.000956,1.201174,AAPL,0.000378,1.263046e-04,6.633264e-08
61957,2025-03-31 23:56:00,221.1001,0.000334,-0.000007,SHORT,-0.000334,1.200772,AAPL,0.000377,1.666262e-04,6.633264e-08
61958,2025-03-31 23:56:00,189.1488,-0.000088,-0.000004,SHORT,0.000088,1.341257,AMZN,0.000624,-1.736718e-04,2.499996e+00
61959,2025-03-31 23:56:00,153.5400,0.000073,-0.000040,SHORT,-0.000073,1.168816,GOOGL,0.000228,-1.343829e-04,6.411771e-08


In [41]:
run_sensitivity_analysis(final_df)

,Adjustment_Factor,Final_Cumulative_Return
0,0.5,1.303457
1,0.8,1.303457
2,0.9,1.303457
3,1.0,1.303457
4,1.1,1.303457
5,1.2,1.303457
6,1.5,1.303457


In [42]:
run_stress_testing(final_df)

,Scenario,Final_Cumulative_Return
0,Crash,16.645232
1,Volatility_Spike,1.301176
2,Prediction_Inverted,0.682761
3,Zero_Prediction,1.071821
4,Normal,1.303457
